In [1]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz
%load_ext line_profiler
# turn off autoreload so that we can use the old model 
# when editing the current project

from coeditor.common import *
import os
from coeditor.model import RetrievalEditorModel, AttentionMode, DecodingArgs
from coeditor.service import EditPredictionService, ChangeDetector
from coeditor.c3problem import C3GeneratorCache, C3Problem

os.chdir(proj_root())

In [2]:
# target_dir = proj_root() / "../SPOT-copy/"
target_dir = proj_root()
model_name = "coeditor-xl-c3-dropout-v1.6-resumed"
model_path = get_model_dir(True) / model_name
model = RetrievalEditorModel.load(model_path)
model.to("cuda:2")
detector = ChangeDetector(target_dir)
service = EditPredictionService(
    detector,
    model,
    dec_args = DecodingArgs(do_sample=False, num_beams=1, length_penalty=0.0)
    # dec_args = DecodingArgs(do_sample=True, top_p=0.95, marginalize_samples=50)
)


In [3]:
# service.dec_args = DecodingArgs(do_sample=True, top_p=0.9, marginalize_samples=25)
# service.dec_args = DecodingArgs(do_sample=False, num_beams=8, length_penalty=0.0)
service.tlogger.clear()
response = service.suggest_edit(to_rel_path("scripts/train_model.py"), 91)
print(response)
service.tlogger.as_dataframe()

Target module 'scripts.train_model' has not changed.
Target span has not changed. Creating a trivial change.


Analyzing /home/jiayi/Projects/Coeditor/scripts/train_model.py: 100%|██████████| 230/230 [00:01<00:00, 134.63it/s]


Writing logs to: .coeditor_logs
Target file: /home/jiayi/Projects/Coeditor/scripts/train_model.py
Edit range: (76, 0) - (106, 0)
Target lines: 76--105
	--------------- Suggestion 0 (score: 1) ---------------
	         model = RetrievalEditorModel.from_code_t5("base", reuse_embed=True)
	     else:
	         model = RetrievalEditorModel.load(resumed_from)
 
	     if os.getenv("CUDA_VISIBLE_DEVICES") is None:
	         warnings.warn(
	             "CUDA_VISIBLE_DEVICES not set, using 0. Note that "
	             "the Huggingface Trainer will use all visible GPUs for training."
	         )
	         os.environ["CUDA_VISIBLE_DEVICES"] = "0"
 
	     train_tkn = encoder.edit_tokenizer
	     eval_tkn = copy.deepcopy(train_tkn)
	     eval_tkn.max_query_tks *= 2
	     eval_tkn.max_output_tks *= 2
	     eval_tkn.max_ref_tks_sum *= 2
 
	     eval_loader = C3DataLoader(
	         datasets["valid"], None, eval_tkn, eval_batch_args, shuffle=False, desc="eval"
	     )
 
	     if not eval_only and resu

,name,count,avg_time,total_time
1,get c3 problem,1,3.210330,3.210330
0,usage analysis,1,2.850747,2.850747
6,run model,1,1.282566,1.282566
4,model.generate,1,1.275702,1.275702
3,decoder.forward,37,0.015015,0.555560
2,tokenize c3 problem,1,0.051294,0.051294
5,assemble changes,1,0.003711,0.003711


In [4]:
testset: list[C3Problem] = pickle_load(get_dataset_dir("xl") / "transformed" / "C3ProblemGenerator(VERSION=2.9)-C3ProblemChangeDropout(VERSION=1.3)")["test"]
len(testset)

39477

In [5]:
subset = testset[0:-1:40]
len(subset)

987

In [6]:
tokenizer = service.c3_tkn
dec_args = service.dec_args
dec_args.num_beams = 1

multi_stats = [model.multi_round_edit_gain(ex, tokenizer, dec_args, print_steps=False) for ex in tqdm(subset, smoothing=0.0)]

100%|██████████| 987/987 [42:17<00:00,  2.57s/it]


In [18]:
keys = ["label_edit_gain", "first_edit_gain", "total_edit_gain", "rounds"]
stats = {k: scalar_stats([getattr(s, k) for s in multi_stats]) for k in keys}   
pretty_print_dict(stats)

print(f"For all edits (n={len(multi_stats)}):")
label_sum = sum(s.label_edit_gain for s in multi_stats)
single_sum = sum(s.first_edit_gain for s in multi_stats)
multi_sum = sum(s.total_edit_gain for s in multi_stats)
print(f"Single-round Gain ratio: {single_sum / label_sum:.2%}")
print(f"Multi-round Gain ratio: {multi_sum / label_sum:.2%}")

big_stats = [s for s in multi_stats if 100 <= s.label_edit_gain <= 500]
print(f"For big edits (n={len(big_stats)}):")
label_sum = sum(s.label_edit_gain for s in big_stats)
single_sum = sum(s.first_edit_gain for s in big_stats)
multi_sum = sum(s.total_edit_gain for s in big_stats)
print(f"Single-round Gain ratio: {single_sum / label_sum:.2%}")
print(f"Multi-round Gain ratio: {multi_sum / label_sum:.2%}")

label_edit_gain:
   mean: 100.08
   median: 40
   min: 0
   max: 2778
first_edit_gain:
   mean: 33.571
   median: 10
   min: 0
   max: 910
total_edit_gain:
   mean: 44.285
   median: 17
   min: -170
   max: 927
rounds:
   mean: 2.075
   median: 1
   min: 1
   max: 6
For all edits (n=987):
Single-round Gain ratio: 33.54%
Multi-round Gain ratio: 44.25%
For big edits (n=245):
Single-round Gain ratio: 32.19%
Multi-round Gain ratio: 45.69%


In [19]:
# find the first index where the edit gain is less than 0
ex_id = next(i for i, s in enumerate(multi_stats) if s.total_edit_gain < 0)
# ex_id = next(i for i, s in enumerate(multi_stats) if s.label_edit_gain > 1000)

In [20]:
ex = subset[ex_id]
ex.print()

--------------------------------------------------------------------------------
summary:
path: ghost/cli
project: EntySec?Ghost
commit: CommitInfo(hash='4e25989af187bf9fcb2f3c4d208155a442aa4632', parents=('d480c76b551b6309850059e75bceb2ec24feb252',), msg='Update __init__.py')
--------------------------------------------------------------------------------
main change:
<add>     """ Ghost Framework command-line interface.
 <add> 
 <add>     :return None: None
 <add>     """
 <add> 
    console = Console()
    console.shell()

--------------------------------------------------------------------------------
edit_line_ids:
(0, 1, 2, 3)


In [21]:
from coeditor.encoding import tokens_to_change, show_change, decode_tokens
from dataclasses import replace
from coeditor.tk_array import TkArray

problem = ex
tokenizer = service.c3_tkn
dec_args = service.dec_args
dec_args.num_beams = 1

model.multi_round_edit_gain(problem, service.c3_tkn, dec_args, print_steps=True)

--------------------------------------------------------------------------------
gold_change:
Modified: 
  +     """ Ghost Framework command-line interface.
  +
  +     :return None: None
  +     """
  +
        console = Console()
        console.shell()
 
Remaining changes:
TkDelta(
  0: ('<add>     """ Ghost Framework command-line interface.\r', '<add> \r', '<add>     :return None: None\r')
  1: ('<add> \r',)
)
insert lines: (0,3), cost=75
insert lines: (4,5), cost=4
total cost: 79
--------------------------------------------------------------------------------
round:
1
--------------------------------------------------------------------------------
path: ghost/cli
n_references: 1
total_reference_tks: 12
project: EntySec?Ghost
commit: CommitInfo(hash='4e25989af187bf9fcb2f3c4d208155a442aa4632', parents=('d480c76b551b6309850059e75bceb2ec24feb252',), msg='Update __init__.py')
========Ground Truth========
 <0>:<add>     """ Ghost Framework command-line interface.
     <add> 
     <add> 

MultiRoundEditStats(label_edit_gain=79, first_edit_gain=0, total_edit_gain=-2, rounds=4)